In [ ]:
import json
import uuid
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd


Import Autodesk's CSV, then sanitize the column names.

In [ ]:
csv_inpath = Path("data/input/autodesk-jira-tickets.csv")
df = pd.read_csv(csv_inpath)
renames = {
    "Generative AI Tool ": "generative_ai_tool",  # Customer data has extraneous space.
    "ticket number": "ticket_number",
    "jira ticket": "jira_ticket",
    "Status": "status",
    "Prompts": "prompts",
    "Purpose of Use": "purpose_of_use",
    "Response to Request": "response_to_request",
}
df.rename(columns=renames, inplace=True)
df.head()

Iterate down the rows, transforming each one into a JSON object defined by [`use-case-schema.json`](./use-case-schema.json).

In [ ]:
# Status mapping
status_mapping = {"Approved": 1, "Pending": 0, "Rejected": -1}


use_cases = []
now_iso = datetime.now(timezone.utc).isoformat()

for _, row in df.iterrows():
    use_case = {
        "id": str(uuid.uuid4()),
        "description": (
            f"Purpose of Use:\n{row['purpose_of_use']}\n\n"
            f"Response to Request:\n{row['response_to_request']}\n\n"
            f"Prompts:\n{row['Prompts ']}\n\n"
            f"Ticket Number: {row['ticket_number']}\n"
            f"JIRA Ticket: {row['jira_ticket']}"
        ),
        "name": row["generative_ai_tool"].strip(),
        "icon": None,
        "ai_type": "",
        "governance_status": status_mapping.get(row["status"].strip(), 0),
        "domains": [],
        "industries": [],
        "regions": [],
        "risk_category_level": None,
        "custom_fields": [],
        "questionnaires": [],
        "inserted_at": now_iso,
        "updated_at": now_iso,
    }
    use_cases.append(use_case)

# Save new version to JSON.
outfolder = csv_inpath.parents[1] / "output"
outfolder.mkdir(parents=True, exist_ok=True)
filename = csv_inpath.with_name(csv_inpath.stem + "-bulk-upload.json").name
json_outpath = outfolder / filename
with open(json_outpath, "w") as f:
    json.dump(use_cases, f, indent=4)

json_outpath